# Tempe Dataset notebook

### Downloading Dependencies

In [5]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install os
!{sys.executable} -m pip install math
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install copy
!{sys.executable} -m pip install imp
!{sys.executable} -m pip install cv2
!{sys.executable} -m pip install json
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pandas

# If gives error, try "brew install boost"
!{sys.executable} -m pip install osm2gmns

### Importing Dependencies

In [46]:
import imp
import os
import numpy as np;
import requests
mapSegUtils = imp.load_source('mapSegmentUtils.py', '../Utils/mapSegmentUtils.py');
osmUtils = imp.load_source('osmUtils.py', '../Utils/osmUtils.py');
imageLinkUtils = imp.load_source('imageLinkUtils.py', '../Utils/imageLinkUtils.py');
imUtils = imp.load_source('imageUtils.py', '../Utils/imageUtils.py');
import math
import matplotlib.pyplot as plt

In [9]:
# Parameters
zoom = 20
imgSize = 400

latTop = 33.4365
latBottom = 33.32759
longLeft = -111.9784
longRight = -111.89361

dirName = "../Dataset/DataSetTempe"
areaName = "Tempe City"
dirName = "../Dataset/DataSetTempe"

In [15]:
n = latTop;
s = latBottom;
w = longLeft;
e = longRight
print("w, s, e, n")
print(w, s, e, n)

# url = "https://www.openstreetmap.org/api/0.6/map?bbox={}%2C{}%2C{}%2C{}".format(w, s, e, n)
url = f"http://overpass.openstreetmap.ru/cgi/xapi_meta?*[bbox={w},{s},{e},{n}]";


print("url:")
print(url)
response = requests.get(url)
if response.status_code == 200:
    file_path = "./map2" + ".osm"
    with open(file_path, 'wb') as file:
        file.write(response.content)
else:
    file_path = 'None'
    print("Failed to retrieve the data.")

w, s, e, n
-111.9784 33.32759 -111.89361 33.4365
url:
http://overpass.openstreetmap.ru/cgi/xapi_meta?*[bbox=-111.9784,33.32759,-111.89361,33.4365]


### Extracting links from OSM data

In [17]:
net = osmUtils.getOsmNodeLinksUsingExistingMap("mapTempe.osm")

In [18]:
osmUtils.outputNetToCSV(net) #net

In [19]:
osmUtils.showNet(net)

### Creating dictionary of image indices to roads present in the image
Like in calculating shade rate, here too we can use link coordinates to find roads that are present in an image.
However, it might not make that big of a difference.

In [36]:
imgFolderPath = "../Dataset/DataSetTempe";
imgDirPath = os.path.join(imgFolderPath, "imageDirectory");

nodeFilePath = "node.csv";
linkFilePath = "link.csv";

# The dictionary
with open('imageLinksDir.pkl', 'rb') as f:
    imageLinksDir = pickle.load(f)

### Finding total shades for each of the road


In [49]:
imUtils.getImageDirData(imgDirPath)
dirPath = imgFolderPath;
nodeFilePath = "node.csv";
linkFilePath = "link.csv";
threshold = 70
totalRoadShadeCoverage, totalMasks, totalImage = imageLinkUtils.calcShadeRatesForRoadsInAllImages(dirPath, nodeFilePath, linkFilePath, threshold, imageLinksDir)

In [ ]:
imgData = imUtils.getImageDirData(imgDirPath);
xLim = imUtils.getImageDirData()['numRows'];
yLim = imUtils.getImageDirData()['numCols'];

### Data Visualization

In [50]:
nodeFilePath = "node.csv";
node2CoordDict = osmUtils.getNode2CoordinatesDict(nodeFilePath)

In [51]:
linkFilePath = "link.csv";
linkID2NodeDict = osmUtils.getLinkID2EndNodesDict(linkFilePath);

In [52]:
linkFilePath = "link.csv";
linkIDToNodePair = osmUtils.getLinkID2EndNodesDict(linkFilePath)

In [53]:
linkFilePath = "link.csv";
nodes2LinkCoordsDict = osmUtils.getNodes2LinkCoordsDict(linkFilePath)

In [ ]:
coordinateSamples = [];
for linkID in totalRoadShadeCoverage:
    # Obtaining link coords from nodes.
    nodePair0 = linkIDToNodePair[linkID]
    linkCoords = nodes2LinkCoordsDict[nodePair0];
    
    for i in range(len(linkCoords) - 1):
    
        percentageCover = totalRoadShadeCoverage[linkID]["percentageCover"];
        node0Coords = linkCoords[i];
        node1Coords = linkCoords[i + 1];


        lat_1 = node0Coords[0];
        long_1 = node0Coords[1];  

        lat_2 = node1Coords[0];
        long_2 = node1Coords[1];

        alpha = np.linspace(0, 1, 50*(math.floor(percentageCover) + 10));

        longSamples = long_1 + alpha * (long_2 - long_1);
        latSamples  = lat_1  + alpha * (lat_2  - lat_1);
        coordSamples = [];
        n = 20;
        
        # Adding samples to thicken the line
        for i in range(len(longSamples)):
            muLat = latSamples[i];
            muLong = longSamples[i];
            # Vary this to vary width - keep around 0.00001
            sigma = 0.00003
            latPerpSamples = np.random.normal(muLat, sigma, n);
            longPerpSamples = np.random.normal(muLong, sigma, n);
            
            addent = np.transpose(np.vstack((latPerpSamples, longPerpSamples)))   
            
            if len(coordSamples) == 0:
                coordSamples = addent;
            else:
                coordSamples = np.vstack((coordSamples, addent));

    
        addent = np.transpose(np.vstack((latSamples, longSamples)))   
                
        if len(coordinateSamples) == 0:
            coordinateSamples = addent;
        else:
            coordinateSamples = np.vstack((coordinateSamples, addent));
            
        coordinateSamples = np.vstack((coordinateSamples, coordSamples));
print(coordinateSamples.shape)

In [ ]:
import datashader as ds, pandas as pd, colorcet
from datashader.colors import Hot
from datashader.colors import viridis

df  = pd.DataFrame(coordinateSamples, columns = ['lat', 'long']);
cvs = ds.Canvas(plot_width=700, plot_height=700)
agg = cvs.points(df, 'lat', 'long')
img = ds.tf.shade(agg, cmap=['white', 'yellow', 'orange', 'red', 'maroon', 'brown'], how='log')
img

In [ ]:
plt.imshow(img)